In [1]:
import numpy as np
import pandas as pd
import os
import random
from PIL import Image, ImageDraw
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torchvision
from torchvision import transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# /content/drive/MyDrive/Helipad
import sys
# FOLDERNAME = "/Users/jamesau/Library/CloudStorage/GoogleDrive-jamesau2810@gmail.com/My Drive/Painting_data"
FOLDERNAME = "Helipad"
# FOLDERNAME = "Colab Notebooks/assignment3"
assert FOLDERNAME is not None, "[!] Enter the foldername."
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
# FOLDERNAME2 = "ust_paint_sample_photo"
# # FOLDERNAME = "Colab Notebooks/assignment3"
# assert FOLDERNAME2 is not None, "[!] Enter the foldername."
# sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME2))

In [4]:
loc = "/content/drive/MyDrive/Helipad/train/"
csv_loc = loc + "_annotations.csv"
# train = pd.read_csv(r"../input/global-wheat-detection/train.csv")
train = pd.read_csv(csv_loc)
train.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,0524heli012_jpg.rf.15c819427aee9f3818fa407173d...,1600,1199,helipad,632,390,759,669
1,0524heli021_jpg.rf.15bc53e62e9e657b27cfec0be28...,480,360,helipad,141,177,289,249
2,516heli013_jpg.rf.2e84c68af0bd1ddabbf0ee44e6bd...,256,192,helipad,22,98,192,135
3,0524heli009_jpg.rf.1c36cfdd795d22436ff3ecc70ae...,1178,804,helipad,533,27,1072,804
4,0524heli023_jpg.rf.26f1c0abdd2c4533228ce5e3d57...,768,512,helipad,201,6,442,512


In [5]:
# torch.cuda.set_per_process_memory_fraction(0.9,0)

In [6]:
train.dtypes

filename    object
width        int64
height       int64
class       object
xmin         int64
ymin         int64
xmax         int64
ymax         int64
dtype: object

In [7]:
train.loc[:,"height"]

0      1199
1       360
2       192
3       804
4       512
       ... 
106    1363
107     463
108     640
109     427
110     650
Name: height, Length: 111, dtype: int64

In [8]:
train.drop(columns=["width","height","class"], inplace = True)
# train.drop(["width","height","class"], axis = 1,inplace = True)

# Image Name Label
unique_imgs = train["filename"].unique()

In [9]:
# train.head()
df = train
df.head()

,filename,xmin,ymin,xmax,ymax
0,0524heli012_jpg.rf.15c819427aee9f3818fa407173d...,632,390,759,669
1,0524heli021_jpg.rf.15bc53e62e9e657b27cfec0be28...,141,177,289,249
2,516heli013_jpg.rf.2e84c68af0bd1ddabbf0ee44e6bd...,22,98,192,135
3,0524heli009_jpg.rf.1c36cfdd795d22436ff3ecc70ae...,533,27,1072,804
4,0524heli023_jpg.rf.26f1c0abdd2c4533228ce5e3d57...,201,6,442,512


In [10]:
unique_imgs

array(['0524heli012_jpg.rf.15c819427aee9f3818fa407173d15774.jpg',
       '0524heli021_jpg.rf.15bc53e62e9e657b27cfec0be282555d.jpg',
       '516heli013_jpg.rf.2e84c68af0bd1ddabbf0ee44e6bd4679.jpg',
       '0524heli009_jpg.rf.1c36cfdd795d22436ff3ecc70aefd438.jpg',
       '0524heli023_jpg.rf.26f1c0abdd2c4533228ce5e3d57220ef.jpg',
       '516heli003_jpg.rf.06ae963a3a9a2a3c6384589b52a3c626.jpg',
       '0524heli010_jpg.rf.3d603d43cd83db2ac688e3f74c303a78.jpg',
       '0524heli012_jpg.rf.27d99b8733c3d23f375d800c48bb6016.jpg',
       '516heli018_jpg.rf.1768e85cc96f1d23134daf68aa8eb620.jpg',
       '516heli014_jpg.rf.37322af1b9573863b6f5e1c2cd2be71c.jpg',
       '516heli015_jpg.rf.227f2083fdde92bed52ba38a2cceafb7.jpg',
       '516heli012_jpg.rf.163ed5052af64ebb4c7fb334b7cff655.jpg',
       '516heli006_jpg.rf.256c735a5873f469254571a86f79be2d.jpg',
       '516heli020_jpg.rf.350ad82d7b07f0fd9dcd0bc49ff1f3a7.jpg',
       '0524heli020_jpg.rf.0fa13ad6662bee92825fe9f39c5e5507.jpg',
       '0524heli02

In [11]:
class CustDat(torch.utils.data.Dataset):
    def __init__(self, df,unique_imgs, indices,ep = "",Prior = "",resize = (-1,-1)):
        self.df = df
        self.unique_imgs = unique_imgs
        self.indices = indices
        self.ep = ep
        self.Prior = Prior
        # self.resize_dim = resize
    def __len__(self):
       return len(self.indices)
    def __getitem__(self, idx):
       image_name = self.unique_imgs[self.indices[idx]]
       #  boxes = self.df[self.df.image_id==image_name].values[:,1:].astype("float")
       boxes = self.df[self.df.filename==image_name].values[:,1:].astype("float")
       link = ""
       link += self.Prior
       link += image_name
       link += self.ep
       #  if self.ep:
           #  link = "../input/global-wheat-detection/train/"+image_name+".jpg"

       img = Image.open(link).convert('RGB')
       # if self.resize_dim != (-1,-1):
       #      img = img.resize(self.resize_dim)
       labels = torch.ones((boxes.shape[0]),dtype = torch.int64)
       target = {}
       target["boxes"] = torch.tensor(boxes)
       target["label"] = labels
       return T.ToTensor()(img), target

In [12]:
train_inds , val_inds = train_test_split(range(unique_imgs.shape[0]),test_size = 0.1)
def custom_collate(data):
   return data

In [13]:
train_dl = torch.utils.data.DataLoader(CustDat(df,unique_imgs,train_inds,"",loc,(640,640)),
                                       batch_size = 8,
                                       shuffle = True,
                                       collate_fn = custom_collate,
                                       pin_memory = True if torch.cuda.is_available() else False
)
val_dl = torch.utils.data.DataLoader(CustDat(df,unique_imgs,train_inds,"",loc,(640,640)),
                                       batch_size = 4,
                                       shuffle = True,
                                       collate_fn = custom_collate,
                                       pin_memory = True if torch.cuda.is_available() else False
)

In [14]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrain = True)
num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 79.6MB/s]


In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001,momentum=0.9,weight_decay =0.0005)
num_epochs = 15

cpu


In [16]:
model.to(device)
for epochs in range(num_epochs):
    epoch_loss = 0
    print("Epoch Number: ", epochs)
    for data in train_dl:
        imgs = []
        targets = []
        for d in data:
            imgs.append(d[0].to(device))
            targ = {}
            targ["boxes"] = d[1]["boxes"].to(device)
            targ["labels"] = d[1]["label"].to(device)
            targets.append(targ)
            # del targ
            # del d
        # print(np.shape(d[0]))
        loss_dict = model(imgs, targets)
        # del targets
        # del imgs
        loss = sum(v for v in loss_dict.values())
        print("Loss: ",loss.cpu().detach().numpy())
        epoch_loss += loss.cpu().detach().numpy()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # del loss
    # gc.collect()
    # torch.cuda.empty_cache()
print(epoch_loss)

Epoch Number:  0
torch.Size([3, 667, 1020])


KeyboardInterrupt: ignored

In [ ]:
model.eval()
data = iter(val_dl).__next__()
# data

In [ ]:
# img = data[0][0]
# boxes = data[0][0]['boxes']
# label = data[0][0]['label']
pic_Num = 0

img = data[pic_Num][0]
boxes = data[pic_Num][1]['boxes']
label = data[pic_Num][1]['label']
print(img)
print(boxes)
print(label)

In [ ]:
IOU_score = 0.45
# IOU_score = 0.7

output = model([img.to(device)])
output
out_bbox = output[0]["boxes"]
out_scores = output[0]["scores"]
keep = torchvision.ops.nms(out_bbox,out_scores,IOU_score)

In [ ]:
im = (img.permute(1,2,0).cpu().detach().numpy()*255).astype('uint8')
im
vsample = Image.fromarray(im)
draw = ImageDraw.Draw(vsample)
# for box in boxes:
#     draw.rectangle(list(box),fill = None,outline = "red")

boxes2 = out_bbox[keep[0],:]
# boxes2 = out_bbox[keep,:]
draw.rectangle(list(boxes2),fill = None,outline = "red")
# print(boxes2)
# for box in boxes2:
#     draw.rectangle(list(box),fill = None,outline = "red")

In [ ]:
img_file = Image.open("/content/drive/MyDrive/Helipad/train/0524heli001_jpg.rf.10b7b46eee8aedb983309f11da43d9df.jpg")
# img_file = Image.open("/content/drive/MyDrive/Painting_data/test/Img109_jpg.rf.0ff2b6d58112c33152e423368f4a507d.jpg")
img_file = img_file.resize((640,640))
img_Unorm = torchvision.transforms.functional.pil_to_tensor( img_file)

In [ ]:
img = torch.div(img_Unorm,256)
IOU_score = 0.45
# IOU_score = 0.7

output = model([img.to(device)])
output
out_bbox = output[0]["boxes"]
out_scores = output[0]["scores"]
keep = torchvision.ops.nms(out_bbox,out_scores,IOU_score)
im = (img.permute(1,2,0).cpu().detach().numpy()*255).astype('uint8')
vsample = Image.fromarray(im)
draw = ImageDraw.Draw(vsample)
# for box in boxes:
#     draw.rectangle(list(box),fill = None,outline = "red")

boxes2 = out_bbox[keep[0],:]
# boxes2 = out_bbox[keep,:]
draw.rectangle(list(boxes2),fill = None,outline = "red")


In [ ]:
vsample

In [ ]:
boxes2

In [ ]:
# Clear cuda Memory
import gc
gc.collect()
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# img_file = Image.open("/content/drive/MyDrive/Helipad/train/0524heli008_jpg.rf.9dd542c8a8024e6e052432e044d04b4c.jpg")
# img_file = img_file.resize((640,640))
# img_file

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Helipad_Detect_model.pth')

In [ ]:

model.load_state_dict

In [ ]:
#匯入模組 - Import library
# import RPi.GPIO as GPIO
import time
import torch
# from picamera2 import Picamera2, Preview
from time import sleep
import numpy as np
from PIL import Image , ImageDraw
import cv2
from torchvision import models, transforms
import torchvision
img_path = '/home/pi/Desktop/image_new_2.jpg'
MODEL_PATH = '/content/drive/MyDrive/Helipad_Detect_model.pth'
# MODEL_PATH = '/home/jamesau/Downloads/Most_ardu_item/Helipad_Detect_model.pth'
# Read the contents of the .txt file
# with open('imagenet1000_clsidx_to_labels.txt', 'r') as file:
#     content = file.read()

# Remove leading/trailing white spaces
# content = content.strip()
IOU_score = 0.45
# Convert the string to a dictionary
# classes = eval(content)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# picam2 = Picamera2()
# camera_config = picam2.create_preview_configuration()
# picam2.configure(camera_config)
# picam2.start_preview(Preview.QTGL)
# picam2.start()
# time.sleep(2)
# picam2.capture_file(img_path)
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 224)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 224)
cap.set(cv2.CAP_PROP_FPS, 36)

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
model_Data = torch.load(MODEL_PATH, map_location=device)
net = models.detection.fasterrcnn_resnet50_fpn(pretrain = True)
num_classes = 2


in_features = net.roi_heads.box_predictor.cls_score.in_features
net.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
net.load_state_dict(model_Data)
net.eval()
# net = models.quantization.mobilenet_v2(pretrained=True)
# jit model to take it from ~20fps to ~30fps
# net = torch.jit.script(net)

started = time.time()
last_logged = time.time()
frame_count = 0
# with torch.no_grad():
def run():
    while True:
        # read frame
        # img_file = Image.open("/content/drive/MyDrive/Helipad/train/0524heli001_jpg.rf.10b7b46eee8aedb983309f11da43d9df.jpg")
        image = cv2.imread("/content/drive/MyDrive/Helipad/train/0524heli001_jpg.rf.10b7b46eee8aedb983309f11da43d9df.jpg")
        # ret, image = cap.read()
        print("Capture One Image")
        # if not ret:
        #     raise RuntimeError("failed to read frame")

        # convert opencv output from BGR to RGB
        image = image[:, :, [2, 1, 0]]
        permuted = image

        # preprocess
        input_tensor = preprocess(image)

        # create a mini-batch as expected by the model
        # input_batch = input_tensor.unsqueeze(0)
        input_batch = input_tensor
        # run model
        net.to(device)
        output = net([input_batch.to(device)])
        out_bbox = output[0]["boxes"]
        out_scores = output[0]["scores"]
        keep = torchvision.ops.nms(out_bbox,out_scores,IOU_score)
        im = (input_batch.permute(1,2,0).cpu().detach().numpy()*255).astype('uint8')
        vsample = Image.fromarray(im)
        draw = ImageDraw.Draw(vsample)
        # for box in boxes:
	#     draw.rectangle(list(box),fill = None,outline = "red")
        boxes2 = out_bbox[keep[0],:]
        # boxes2 = out_bbox[keep,:]
        draw.rectangle(list(boxes2),fill = None,outline = "red")
        # output = net(input_batch)
        print(output)
        # opencvImage = np.array(vsample)
        vsample.show()
        # opencvImage = cv2.cvtColor(np.array(vsample), cv2.COLOR_RGB2BGR)
        # cv2.imshow('image window', opencvImage)
        # add wait key. window waits until user presses a key
        # cv2.waitKey(0)
        # and finally destroy/close all open windows
        # cv2.destroyAllWindows()
        # # do something with output ...
        # top = list(enumerate(output[0].softmax(dim=0)))
        # top.sort(key=lambda x: x[1], reverse=True)
        # idx, val = top[0]
        # # for idx, val in top[:10]:
        # print(f"{val.item() * 100:.2f}% {classes[idx]}")
		# # print(f"{val.item()*100:.2f}% {idx}")
        # # log model performance
        # # frame_count += 1
        # now = time.time()
        # if now - last_logged > 1:
            # print(f"{frame_count / (now-last_logged)} fps")
            # last_logged = now
            # frame_count = 0
run()
# ret, image = cap.read()
# # convert opencv output from BGR to RGB
# image = image[:, :, [2, 1, 0]]
# cv2.imwrite(img_path, image)
# img1 = Image.fromarray(image)
# img1.save(img_path)

# camera = Picamera2()
# print(camera.resolution)
# output = np.empty((768, 1024, 3), dtype=np.uint8)
# camera.start_preview()
# sleep(5)
# camera.capture(output, 'rgb')
# # camera.capture(img_path)
# camera.stop_preview()
# img = Image.fromarray(output, "RGB")
# img.save(img_path)
#Pin變數 - Pin variable
# LED_PIN = 5
#
# #設置GPIO - GPIO Setup
# GPIO.setmode(GPIO.BCM) #GPIO.BCM (GPIO引腳號) / GPIO.BOARD (物理引腳號)
# GPIO.setup(LED_PIN, GPIO.OUT)
# GPIO.output(LED_PIN, 1)
# time.sleep(0.1)
# GPIO.output(LED_PIN, 0)
#無限循環
#while True:
	#輸出高電平 High
	#GPIO.output(LED_PIN, 1)
	#print(1)
	#卡著等待0.1秒，再繼續執行下一行
	#time.sleep(0.1)
	#輸出低電平 Low
	#GPIO.output(LED_PIN, 0)
	#卡著等待0.1秒，再繼續循環
	#time.sleep(0.1)
